In [1]:
!wget https://s3.amazonaws.com/text-datasets/imdb_full.pkl
!wget https://s3.amazonaws.com/text-datasets/imdb_word_index.json

--2020-01-13 10:37:15--  https://s3.amazonaws.com/text-datasets/imdb_full.pkl
Risoluzione di s3.amazonaws.com (s3.amazonaws.com)... 52.216.112.197
Connessione a s3.amazonaws.com (s3.amazonaws.com)|52.216.112.197|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 65552540 (63M) [application/octet-stream]
Salvataggio in: "imdb_full.pkl"

imdb_full.pkl       100%[===================>]  62,52M  9,87MB/s    in 7,2s    

2020-01-13 10:37:23 (8,62 MB/s) - "imdb_full.pkl" salvato [65552540/65552540]

--2020-01-13 10:37:23--  https://s3.amazonaws.com/text-datasets/imdb_word_index.json
Risoluzione di s3.amazonaws.com (s3.amazonaws.com)... 52.216.206.189
Connessione a s3.amazonaws.com (s3.amazonaws.com)|52.216.206.189|:443... connesso.
Richiesta HTTP inviata, in attesa di risposta... 200 OK
Lunghezza: 1641221 (1,6M) [application/octet-stream]
Salvataggio in: "imdb_word_index.json"

imdb_word_index.jso 100%[===================>]   1,56M  2,36MB/s    in 0,7s    

2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import pickle
data = pickle.load(open('imdb_full.pkl', 'rb'))

import json
vocab = json.load(open('imdb_word_index.json'))

In [3]:
inv = {idx:word for word, idx in vocab.items()}

In [4]:
(X_train, y_train), (Xt, yt) = data

In [5]:
trainidx = [i for i, x in enumerate(X_train) if len(x) < 400]
trainidx, devidx = train_test_split(trainidx, train_size=0.8, random_state=1378)
X = [X_train[i] for i in trainidx]
y = [y_train[i] for i in trainidx]

Xd = [X_train[i] for i in devidx]
yd = [y_train[i] for i in devidx]

testidx = [i for i, x in enumerate(Xt) if len(x) < 400]
testidx, remaining_idx =  train_test_split(testidx, train_size=0.2, random_state=1378)

Xt = [Xt[i] for i in testidx]
yt = [yt[i] for i in testidx]

/home/oliviero/miniconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [6]:
def invert_and_join(X) :
    X = [[inv[x] for x in doc] for doc in X]
    X = [" ".join(x) for x in X]
    return X

In [7]:
X = invert_and_join(X)
Xd = invert_and_join(Xd)
Xt = invert_and_join(Xt)

In [8]:
texts = {'train' : X, 'test' : Xt, 'dev' : Xd}
labels = {'train' : y, 'test' : yt, 'dev' : yd}

In [9]:
import pandas as pd
df_texts = []
df_labels = []
df_exp_splits = []

for key in ['train', 'test', 'dev'] :
    df_texts += texts[key]
    df_labels += labels[key]
    df_exp_splits += [key] * len(texts[key])
    
df = pd.DataFrame({'text' : df_texts, 'label' : df_labels, 'exp_split' : df_exp_splits})
df.to_csv('imdb_dataset.csv', index=False)

In [10]:
%run "../preprocess_data_BC.py" --data_file imdb_dataset.csv --output_file ./vec_imdb.p --word_vectors_type fasttext.simple.300d --min_df 10

Vocabulary size :  12487
Found 11522 words in model out of 12487
